# Soft Prompt Tuning

Prompt tuning adds a small set of trainable virtual tokens (soft prompts) to the input while keeping the pre-trained model's weight frozen. 
These virtual prompts are not human-readable; they are appended to the start of a prompt to serve as a task-specific guide during LLM training or inferences.

For example, the virtual tokens are prefixed to text for sentiment classification:

```
[virtual tokens] I love Fridays!
```

where `[virtual tokens]` are the inserted embeddings. These virtual tokens can be randomly generated or initialized from a vocabulary.

During training, these token embeddings are updated while the base model remains frozen. 

In [1]:
# Run the following to downgrade datasets module, restart kernal 
%uv add datasets==3.6.0

/opt/tmp/ellm-2025-nov17/.venv/bin/python: No module named uv
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import packages
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType, PeftConfig, PeftModel
from datasets import load_dataset
from torch.utils.data import DataLoader

from tqdm import tqdm

In [2]:
model_name = "bigscience/bloomz-560m"

username = 'ought/raft'
dataset_name = "twitter_complaints"

In [3]:
# TODO: Load dataset
dataset = load_dataset(username, dataset_name)

Using the latest cached version of the dataset since ought/raft couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'twitter_complaints' at /home/cmlee/.cache/huggingface/datasets/ought___raft/twitter_complaints/1.1.0/9ee50172ea9afda2f1033c6f1b986e568b862fb3 (last modified on Tue Oct 14 16:07:36 2025).


In [4]:
# TODO: Display dataset 
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 3399
    })
})


In [12]:
idx = 20
print(dataset['train'][idx])

{'Tweet text': '@united not happy with this delay from Newark to Manchester tonight :( only 30 mins free Wi-fi sucks ...', 'ID': 20, 'Label': 1}


In [7]:
classes = dataset['train'].features['Label'].names
print(classes)

['Unlabeled', 'complaint', 'no complaint']


In [19]:
# TODO: Add text_label from Label
# Call dataset_enh
dataset_enh = dataset.map(
   lambda x: { 'text_label': [ classes[l] for l in x['Label'] ]},
   batched=True,
   num_proc=1
)

In [20]:
print(dataset_enh['train'][idx])

{'Tweet text': '@united not happy with this delay from Newark to Manchester tonight :( only 30 mins free Wi-fi sucks ...', 'ID': 20, 'Label': 1, 'text_label': 'complaint'}


In [15]:
# TODO: Load tokenizer
# call variable name tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [16]:
# Utility function to preprocess teweets

def preprocess_function(examples):
    text_column = 'Tweet text'
    label_column = 'text_label'
    max_length = 64
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [21]:
# TODO: Preprocess the Tweets
# Call preprocessed tweets dataset_processed
dataset_processed = dataset_enh.map(
   preprocess_function,
   batched=True,
   load_from_cache_file=False    
)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [22]:
# TODO: Explore preprocessed Tweets
print(dataset_processed['train'][idx])

{'Tweet text': '@united not happy with this delay from Newark to Manchester tonight :( only 30 mins free Wi-fi sucks ...', 'ID': 20, 'Label': 1, 'text_label': 'complaint', 'input_ids': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 227985, 5484, 915, 2566, 309, 4904, 1130, 38782, 1002, 1119, 29052, 1485, 5161, 4782, 427, 31451, 83342, 915, 11, 3804, 3171, 173064, 9955, 760, 5279, 10641, 7182, 4443, 2815, 77658, 915, 210, 16449, 5952, 3], 'attention_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 

In [23]:
# TODO: Create train and evaluation dataset 
# Must call train set DataLoader loader_train
# Must call test set DataLoader loader_test

batch_size = 8
dataset_train = dataset_processed['train']
dataset_test = dataset_processed['test']

loader_train = DataLoader(dataset_train, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
loader_test = DataLoader(dataset_test, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)


## Soft Prompt 

In [24]:
# TODO: Create PEFT configuration
config = PromptTuningConfig(
   task_type=TaskType.CAUSAL_LM,
   tokenizer_name_or_path=model_name,
   prompt_tuning_init=PromptTuningInit.RANDOM,
   num_virtual_tokens=8
)

config = PromptTuningConfig(
   task_type=TaskType.CAUSAL_LM,
   tokenizer_name_or_path=model_name,
   prompt_tuning_init=PromptTuningInit.TEXT,
   prompt_tuning_init_text="Big black bug bleeds black blood",
   num_virtual_tokens=8
)


In [25]:
# TODO: Create model for training
base_model = AutoModelForCausalLM.from_pretrained(model_name)
peft_model = get_peft_model(base_model, peft_config=config)

print(peft_model.print_trainable_parameters())

trainable params: 8,192 || all params: 559,222,784 || trainable%: 0.0015
None


In [26]:
print(peft_model)

PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(250880, 1024)
      (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
          )
        )
      

In [ ]:
# Create optimizer
lr = 3e-2
num_epochs = 1
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
   optimizer=optimizer,
   num_warmup_steps=0,
   num_training_steps=(len(loader_train) * num_epochs)
)

In [ ]:
# Train model
# default to CPU
device = "cuda"
device = "cpu"

model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(loader_train)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.requires_grad = True
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(loader_test)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(loader_test)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(loader_train)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

## Inferencing

In [3]:
model_name = "bigscience/bloomz-560m"
peft_model_id = "stevhliu/bloomz-560m_PROMPT_TUNING_CAUSAL_LM"

In [5]:
## TODO: Load trained model
base_model = AutoModelForCausalLM.from_pretrained(model_name)
vprompt_model = PeftModel.from_pretrained(base_model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
print(vprompt_model)

In [7]:
## TODO: Write and encode text

input_text = "Tweet text: @uni I hate the food in your canteen. Label: " 
input_text = "Tweet text: @husband The pipe has been leaking for over one month. Can fix this NOW? Label: "

enc_input = tokenizer(input_text, return_tensors='pt')
print(enc_input)

{'input_ids': tensor([[227985, 106659,   2566,  71187,  29069,   1387,  55907,   1809,   3784,
            578,  17444,    613,   3478,   2592,  13802,     17,   6728,   8256,
           1119, 227284,     34,  77658,     29,    210]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
## TODO: Determine sentiment from model
enc_sentiment = vprompt_model.generate(
   input_ids=enc_input.input_ids,
   attention_mask=enc_input.attention_mask,
   eos_token_id=3
)

In [14]:
sentiment = tokenizer.decode(enc_sentiment[0], skip_special_tokens=True)
print(sentiment)

Tweet text: @husband The pipe has been leaking for over one month. Can fix this NOW? Label: complaint


In [16]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [18]:
enc_prediction = model.generate(
   input_ids=enc_input.input_ids,
   attention_mask=enc_input.attention_mask
)


In [19]:
sentiment = tokenizer.decode(enc_prediction[0])
print(sentiment)

Tweet text: @husband The pipe has been leaking for over one month. Can fix this NOW? Label:  👉</s>
